In [ ]:
import cv2
import numpy as np
import pandas as pd
from google.colab import files
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment
from PIL import Image as PILImage

# Upload images
uploaded = files.upload()

# Face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Make folders
os.makedirs("annotated_images", exist_ok=True)

# Excel setup
wb = Workbook()
ws = wb.active
ws.title = "Face RGB Data"

# Excel styles
header_font = Font(bold=True)
border = Border(
    left=Side(style='thin'), right=Side(style='thin'),
    top=Side(style='thin'), bottom=Side(style='thin')
)
highlight = PatternFill(start_color="FFFACD", end_color="FFFACD", fill_type="solid")
center_align = Alignment(horizontal="center", vertical="center")

# Starting row
current_row = 1

# Loop through uploaded images
for image_name in uploaded:
    print(f"Processing: {image_name}")
    img = cv2.imread(image_name)

    # Header
    ws.cell(row=current_row, column=1, value=f"Image: {image_name}")
    current_row += 1

    if img is None:
        ws.cell(row=current_row, column=2, value="Reason: Invalid image")
        current_row += 2
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) == 0:
        ws.cell(row=current_row, column=2, value="Reason: No face detected")
        current_row += 2
        continue

    # Detect points
    (x, y, w, h) = faces[0]
    forehead = (x + w // 2, y + h // 6)
    left_cheek = (x + w // 4, y + int(h / 1.8))
    right_cheek = (x + 3 * w // 4, y + int(h / 1.8))

    def get_rgb(pt):
        x, y = pt
        b, g, r = img[y, x]
        return int(r), int(g), int(b)

    fr, fg, fb = get_rgb(forehead)
    lr, lg, lb = get_rgb(left_cheek)
    rr, rg, rb = get_rgb(right_cheek)

    # Table header
    ws.append(["", "Part", "R", "G", "B"])
    current_row += 1
    for col in range(2, 6):
        cell = ws.cell(row=current_row - 1, column=col)
        cell.font = header_font
        cell.border = border
        cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")
        cell.alignment = center_align

    # RGB rows
    rgb_rows = [
        ["Forehead", fr, fg, fb],
        ["Left Cheek", lr, lg, lb],
        ["Right Cheek", rr, rg, rb]
    ]

    for row_data in rgb_rows:
        ws.append([""] + row_data)
        for col in range(2, 6):
            cell = ws.cell(row=current_row, column=col)
            cell.border = border
            cell.fill = highlight
            cell.alignment = center_align
        current_row += 1

    # Annotate image
    img_copy = img.copy()
    cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for pt in [forehead, left_cheek, right_cheek]:
        cv2.circle(img_copy, pt, 5, (0, 0, 255), -1)

    # Save and convert to Excel-compatible PNG
    annotated_path = f"annotated_images/annotated_{image_name}.png"
    cv2.imwrite(annotated_path, img_copy)

    pil_img = PILImage.open(annotated_path)
    pil_img.thumbnail((250, 250))  # Resize for Excel
    pil_img.save(annotated_path)

    # Insert image into Excel
    img_excel = ExcelImage(annotated_path)
    img_excel.anchor = f"F{current_row - 3}"  # Insert beside the table
    ws.add_image(img_excel)

    current_row += 2  # Add gap before next image block

# Auto column widths
for col in ["A", "B", "C", "D", "E"]:
    ws.column_dimensions[col].width = 16

# Save Excel
excel_filename = "face_rgb_with_images.xlsx"
wb.save(excel_filename)
files.download(excel_filename)

print("\nExcel downloaded with RGB tables + embedded annotated images.")
